# CONTRATO DE INTEGRACIÓN DS-BACKEND
ChurnInsight - Hackathon NoCountry 2025 - H12-25-L-Equipo 43

## TECNOLOGÍA
- **Framework DS:** FastAPI
- **Puerto:** 8000 (configurable)
- **Endpoint:** POST /predict

---

## REQUEST BODY (lo que Backend envía a DS)
```json
{
  "antiguedad": 18,
  "plan": "premium",
  "metodo_pago": "tarjeta_credito",
  "facturas_impagas": 1,
  "frecuencia_uso": 22,
  "tickets_soporte": 3,
  "tipo_contrato": "mensual",
  "cambios_plan": 1,
  "canal_adquisicion": "web"
}
```

**Todos los campos son obligatorios.**

### Descripción de campos:

- `antiguedad` (int): Meses como cliente (0-120)
- `plan` (string): Tipo de plan ["basico", "estandar", "premium"]
- `metodo_pago` (string): Método de pago ["transferencia_bancaria", "tarjeta_credito", "tarjeta_debito"]
- `facturas_impagas` (int): Cantidad de facturas sin pagar (0-10)
- `frecuencia_uso` (int): Días activos por mes (0-30)
- `tickets_soporte` (int): Cantidad de tickets de soporte-ayuda abiertos (0-50)
- `tipo_contrato` (string): Compromiso temporal ["mensual", "anual"]
- `cambios_plan` (int): Cambios de plan en últimos 6 meses (0-5)
- `canal_adquisicion` (string): Canal de llegada ["web", "referido", "redes_sociales"]

---

## RESPONSE BODY (200 OK) (lo que DS devuelve a Backend)
```json
{
  "prediccion": "no_cancelara",
  "probabilidad_churn": 0.23,
  "umbral_decision": 0.2190,
  "top_features": [
    {
      "feature": "frecuencia_uso",
      "valor_cliente": "22",
      "impacto": "bajo_riesgo",
      "importancia": 0.0245
    },
    {
      "feature": "antiguedad",
      "valor_cliente": "18",
      "impacto": "bajo_riesgo",
      "importancia": 0.0361
    },
    {
      "feature": "facturas_impagas",
      "valor_cliente": "1",
      "impacto": "alto_riesgo",
      "importancia": 0.0854
    }
  ],
  "modelo_version": "v2.0_xgboost_calibrado"
}
```

### Descripción:

- `prediccion` (string): Resultado de clasificación ["cancelara", "no_cancelara"]
- `probabilidad_churn` (float): Probabilidad de cancelación (0.0-1.0)
- `umbral_decision` (float): Umbral usado para la decisión (0.2190)
- `top_features` (array): 3 variables más influyentes según el modelo
  - `feature` (string): Nombre de la variable
  - `valor_cliente` (string): Valor que tiene el cliente
  - `impacto` (string): Nivel de riesgo ["alto_riesgo", "medio_riesgo", "bajo_riesgo"]
  - `importancia` (float): Importancia de la feature en el modelo (0.0-1.0)
- `modelo_version` (string): Versión del modelo (para logs/auditoría)

**NOTA IMPORTANTE:** Los nombres exactos de las top 3 features se definen por el modelo entrenado. Backend debe estar preparado para recibir cualquier combinación de las 9 features de entrada o features engineerizadas.

---

## ERRORES (422 Unprocessable Entity)
```json
{
  "detail": [
    {
      "type": "literal_error",
      "loc": ["body", "plan"],
      "msg": "Input should be 'basico', 'estandar' or 'premium'",
      "input": "Premium",
      "ctx": {
        "expected": "'basico', 'estandar' or 'premium'"
      }
    }
  ]
}
```

---

## INTEGRACIÓN

### Arquitectura:
```
[Frontend] → [Backend Spring Boot :8080] → [Microservicio DS FastAPI :8000] → [Modelo ML]
```

### Flujo:

1. Frontend envía formulario a Backend
2. Backend valida y hace POST a `http://churn-api:8000/predict` (dentro de Docker Compose)
3. DS (FastAPI) carga modelo, predice y devuelve JSON
4. Backend enriquece response (agrega timestamp, guarda en BD) y responde a Frontend

### Responsabilidades:

- **Backend:** Validación de entrada, persistencia, logs, timestamp
- **DS:** Solo predicción ML (sin BD, sin timestamp, sin validaciones de negocio)

---

## CASOS DE PRUEBA

### 1. Cliente de alto riesgo (prob > 0.7):
```json
{
  "antiguedad": 3,
  "plan": "basico",
  "metodo_pago": "transferencia_bancaria",
  "facturas_impagas": 3,
  "frecuencia_uso": 5,
  "tickets_soporte": 8,
  "tipo_contrato": "mensual",
  "cambios_plan": 2,
  "canal_adquisicion": "redes_sociales"
}
```

**Resultado esperado:** `"prediccion": "cancelara"`, `probabilidad_churn > 0.5`

---

### 2. Cliente de bajo riesgo (prob < 0.3):
```json
{
  "antiguedad": 48,
  "plan": "premium",
  "metodo_pago": "tarjeta_credito",
  "facturas_impagas": 0,
  "frecuencia_uso": 28,
  "tickets_soporte": 0,
  "tipo_contrato": "anual",
  "cambios_plan": 0,
  "canal_adquisicion": "referido"
}
```

**Resultado esperado:** `"prediccion": "no_cancelara"`, `probabilidad_churn < 0.3`

---

### 3. Cliente de riesgo medio (prob 0.3-0.7):
```json
{
  "antiguedad": 18,
  "plan": "estandar",
  "metodo_pago": "tarjeta_credito",
  "facturas_impagas": 1,
  "frecuencia_uso": 15,
  "tickets_soporte": 3,
  "tipo_contrato": "mensual",
  "cambios_plan": 1,
  "canal_adquisicion": "web"
}
```

**Resultado esperado:** `"prediccion": "cancelara"` o `"no_cancelara"` (depende del umbral 0.2190)

---

## EJEMPLO DE USO (cURL)

### Desarrollo local:
```bash
curl -X POST http://localhost:8000/predict \
  -H "Content-Type: application/json" \
  -d '{
    "antiguedad": 18,
    "plan": "premium",
    "metodo_pago": "tarjeta_credito",
    "facturas_impagas": 1,
    "frecuencia_uso": 22,
    "tickets_soporte": 3,
    "tipo_contrato": "mensual",
    "cambios_plan": 1,
    "canal_adquisicion": "web"
  }'
```

### Dentro de Docker Compose (desde Backend Java):
```java
RestTemplate restTemplate = new RestTemplate();
String url = "http://churn-api:8000/predict";

ChurnRequest request = ChurnRequest.builder()
    .antiguedad(18)
    .plan("premium")
    .metodoPago("tarjeta_credito")
    .facturasImpagas(1)
    .frecuenciaUso(22)
    .ticketsSoporte(3)
    .tipoContrato("mensual")
    .cambiosPlan(1)
    .canalAdquisicion("web")
    .build();

ResponseEntity<ChurnResponse> response = restTemplate.postForEntity(url, request, ChurnResponse.class);
```

---

## CONSIDERACIONES TÉCNICAS

### ¿Por qué FastAPI?

- Validación automática con Pydantic
- Documentación Swagger en /docs (automática)
- Más rápido que Flask
- Type hints nativos (menos bugs)

### ¿Por qué puerto 8000?

- Convención FastAPI (configurable)
- Backend en 8080, DS en 8000 (separación clara)

### Formato de respuesta

- **prediccion:** Solo dos valores posibles: "cancelara" o "no_cancelara" (sin tildes, minúsculas, con guión bajo)
- **umbral_decision:** Incluido en la respuesta para transparencia (valor: 0.2190)
- **top_features:** Incluye la importancia numérica de cada feature

### Timestamp y confianza

- **NO incluidos** en response DS
- Backend puede agregar timestamp al guardar en BD
- Frontend puede calcular "confianza" de la probabilidad_churn:
  - Si prob < 0.3: "Riesgo Bajo"
  - Si 0.3 ≤ prob ≤ 0.7: "Riesgo Medio"
  - Si prob > 0.7: "Riesgo Alto"

---

## ENDPOINTS ADICIONALES

### GET /health

Health check del microservicio.

**Response (200 OK):**
```json
{
  "status": "healthy",
  "model_loaded": true,
  "model_version": "v2.0_xgboost_calibrado",
  "model_type": "XGBoost + CalibratedClassifierCV"
}
```

### GET /model-info

Información del modelo y métricas.

**Response (200 OK):**
```json
{
  "version": "v2.0_xgboost_calibrado",
  "tipo_modelo": "XGBoost + CalibratedClassifierCV",
  "umbral_decision": 0.2190,
  "metricas": {
    "accuracy": 0.6381,
    "precision": 0.395,
    "recall": 0.8009,
    "f1_score": 0.5291,
    "roc_auc": 0.7688
  },
  "top_3_features": [
    "facturas_impagas",
    "tipo_contrato_mensual",
    "cliente_problematico"
  ],
  "estrategia": "Recall ~80% (priorizar deteccion de churn)"
}
```

---

## DOCUMENTACIÓN AUTOMÁTICA

FastAPI genera documentación interactiva automáticamente:

- **Swagger UI:** http://localhost:8000/docs
- **ReDoc:** http://localhost:8000/redoc
- **OpenAPI JSON:** http://localhost:8000/openapi.json

---

## VALIDACIONES

### Valores válidos por campo:

| Campo | Valores permitidos |
|-------|-------------------|
| plan | "basico", "estandar", "premium" |
| metodo_pago | "tarjeta_credito", "tarjeta_debito", "transferencia_bancaria" |
| tipo_contrato | "mensual", "anual" |
| canal_adquisicion | "web", "referido", "redes_sociales" |

**IMPORTANTE:** Todos los strings deben ser en minúsculas y con guiones bajos (snake_case).

---

## CAMBIOS RESPECTO A VERSIÓN ANTERIOR

1.  Todos los valores de strings en minúsculas y snake_case
2.  Eliminado "efectivo" de metodo_pago (no está en el modelo)
3.  Eliminado "call_center" de canal_adquisicion (no está en el modelo)
4.  Formato de prediccion: "cancelara" / "no_cancelara" (consistente)
5.  Agregado campo umbral_decision en response
6.  Agregado campo importancia en top_features
7.  Actualizado modelo_version a v2.0_xgboost_calibrado
8.  Agregados endpoints adicionales (/health, /model-info)

---

**Versión del contrato:** 2.0  
**Última actualización:** 06/01/2026  
**Equipo:** ChurnInsight - NoCountry H12-25-L-Equipo 43